In [1]:
import pandas as pd
import gensim.downloader as api
import numpy as np

In [2]:
data = pd.read_csv("C:/Users/usrva/OneDrive/Desktop/Projects/Job family classification/Master Job Role (2).csv")
data.head()

,S.No.,Job Role,Job Family,Family Headcount Flag,Description,Skills,Soft Skills,Digitization Quotient,Digitization Quotient Description,Adjacent Families,...,Machine Generated Core Skills,Machine Generated Soft Skills,ONET Code,ONET Code Classification,ONET Link,Deletion Status,Created By,Modified By,Created On,Modified On
0,1,Applications Developer/Engineer,"Software Developers, Applications",True,"The role is responsible for designing, develop...",Python (Programming Language); Hyper Text Mark...,Problem Identification; Coordination; Professi...,15.0,Digitally influenced,"Software Developers, Mobile Applications",...,{},{},15-1252.00,Software Developers,https://www.onetonline.org/link/summary/15-125...,0,kesavanraju,venkatesh.b,NaN,2024-07-01T16:03:22.932374Z
1,2,Backend Engineer,"Software Developers, Applications",False,"The role is responsible for designing, buildin...",Python; Redis; Java; Hadoop; Apache Webserver;...,Cross Functional Problem Solving; Team Handlin...,19.0,Digitally influenced,NaN,...,"[{'name': 'Python', 'count': 51272, 'is_niche'...","[{'name': 'Cross Functional Problem Solving', ...",15-1254.00,Web Developers,https://www.onetonline.org/link/summary/15-125...,0,NaN,venkatesh.b,2021-12-23T13:31:03.179397Z,2024-07-01T16:03:22.948465Z
2,3,Frontend Engineer,"Software Developers, Applications",True,"The role is responsible for designing, buildin...",HTML; CSS; JavaScript; jQuery; Vue.js; Angular...,Time Management; Prioritization; Social Etique...,18.0,Digitally influenced,NaN,...,{},{},15-1254.00,Web Developers,https://www.onetonline.org/link/summary/15-125...,0,NaN,venkatesh.b,2021-12-23T13:31:03.179397Z,2024-07-01T16:03:22.960797Z
3,4,Full Stack Engineer,"Software Developers, Applications",False,The role is responsible for developing and des...,API Development; CSS; Angular.js; React.js; Dj...,Problem Solving; Curiosity; Ideation; Detail O...,12.0,Digitally influenced,NaN,...,{},{},15-1252.00,Software Developers,https://www.onetonline.org/link/summary/15-125...,0,NaN,venkatesh.b,2021-12-23T13:31:03.179397Z,2024-07-01T16:03:22.975985Z
4,5,Android Developer,"Software Developers, Mobile Applications",False,The role is responsible for developing applica...,C(Programming Language); Python (Programming L...,Passionate; Problem Solving; Creativity; Analy...,19.0,Digitally influenced,NaN,...,{},{},15-1252.00,Software Developers,https://www.onetonline.org/link/summary/15-125...,0,NaN,venkatesh.b,2021-12-23T13:31:03.179397Z,2024-07-01T16:03:22.989859Z


In [3]:
job_roles = data['Job Role'].tolist()
job_roles[:10]

['Applications Developer/Engineer',
 'Backend Engineer',
 'Frontend Engineer',
 'Full Stack Engineer',
 'Android Developer',
 'iOS Developer',
 'QA Manual Test Engineer',
 'Salesforce Developer',
 'Java Developer',
 '.NET Developer']

In [4]:
for i in list(api.info()['models']):
    print(i)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [5]:
word2vec_model = api.load("word2vec-google-news-300") 

In [6]:
import re


def avg_vector_of_size_300(role, model):
    role_tokens = re.split(r'[ /,-]', role.upper())
    list_of_vector_for_each_taken = [model[token] for token in role_tokens if token in model]
    if not list_of_vector_for_each_taken:
        return np.zeros(model.vector_size)
    return np.mean(list_of_vector_for_each_taken, axis=0)

In [7]:
job_role_embeddings = np.array([avg_vector_of_size_300(role, word2vec_model) for role in job_roles])

In [8]:
from sklearn.metrics.pairwise import cosine_similarity


def find_most_similar_job_role(new_job_role, job_roles, job_role_embeddings, model):
    new_job_role_embedding = avg_vector_of_size_300(new_job_role, model)
    similarities = cosine_similarity([new_job_role_embedding], job_role_embeddings)
    most_similar_index = np.argmax(similarities)
    return job_roles[most_similar_index]


def find_all_similar_job_roles(new_job_role, job_roles, job_role_embeddings, model):
    new_job_role_embedding = avg_vector_of_size_300(new_job_role, model)
    similarities = cosine_similarity([new_job_role_embedding], job_role_embeddings)[0]
    similarity_list = list(zip(job_roles, similarities))
    sorted_similarity_list = sorted(similarity_list, key=lambda x: x[1], reverse=True)
    return sorted_similarity_list

In [9]:
from scipy.spatial.distance import euclidean


def find_most_similar_job_role_with_euclidean_distance(new_job_role, job_roles, job_role_embeddings, model):
    new_job_role_embedding = avg_vector_of_size_300(new_job_role, model)
    distances = [euclidean(new_job_role_embedding, emb) for emb in job_role_embeddings]
    most_similar_index = np.argmin(distances)
    return job_roles[most_similar_index]


def find_all_similar_job_roles_with_euclidean_distance(new_job_role, job_roles, job_role_embeddings, model):
    new_job_role_embedding = avg_vector_of_size_300(new_job_role, model)
    distances = [euclidean(new_job_role_embedding, emb) for emb in job_role_embeddings]
    similarity_list = list(zip(job_roles, distances))
    sorted_similarity_list = sorted(similarity_list, key=lambda x: x[1])
    return sorted_similarity_list


In [10]:
new_job_role = "IT Support Engineer"
most_similar_job_role = find_most_similar_job_role(new_job_role, job_roles, job_role_embeddings, word2vec_model)
print(most_similar_job_role)

NFVi Support Engineer


In [11]:
most_similar_job_role = find_most_similar_job_role_with_euclidean_distance(new_job_role, job_roles, job_role_embeddings, word2vec_model)
print(most_similar_job_role)

Integration Support Engineer


In [12]:
similar_job_roles = find_all_similar_job_roles(new_job_role, job_roles, job_role_embeddings, word2vec_model)
for role, score in similar_job_roles:
    print(f"{role}----->{score}")

NFVi Support Engineer----->0.9373164268609551
Integration Support Engineer----->0.93115731350446
Customer Support Engineer----->0.9213315996918153
IT Configuration Engineer----->0.9109311574066776
Technical Support Engineer----->0.9059716480568927
IT Engineer----->0.9058442613128832
Network Support Engineer----->0.8960368361484057
IT Support Specialist----->0.8919065828932418
Cloud Support Engineer----->0.88682493868221
IT Storage Engineer----->0.8862408203775998
IT Security Automation Engineer----->0.8841109072787636
Packet Core Support Engineer----->0.8827171283489494
IMS Support Engineer----->0.8787519372617134
IT Infrastructure Engineer----->0.8761318071421399
IT Security Engineer----->0.8734421063673059
IT Security Engineer - Splunk----->0.8734421063673059
CS Core Support Engineer----->0.8666383504971252
Sourcing Engineer----->0.8603486127963518
NFVi Integration Engineer----->0.8573032673897892
Server Engineer----->0.852320486818124
Board Support Package Engineer----->0.8522764373

In [13]:
similar_job_roles = find_all_similar_job_roles_with_euclidean_distance(new_job_role, job_roles, job_role_embeddings, word2vec_model)
for role, score in similar_job_roles:
    print(f"{role}----->{score}")

Integration Support Engineer----->1.0575267682996878
IT Configuration Engineer----->1.110122128334832
Customer Support Engineer----->1.1763062985518613
IT Engineer----->1.1974225408894743
IT Support Specialist----->1.2064670444780363
NFVi Support Engineer----->1.2259509313691819
Packet Core Support Engineer----->1.2376396802951857
IT Security Automation Engineer----->1.2623694321329992
IMS Support Engineer----->1.2681353754749327
IT Storage Engineer----->1.278818130022633
CS Core Support Engineer----->1.3138619396211135
Technical Support Engineer----->1.3170490108268422
Cloud Support Engineer----->1.3316955389644676
IT Infrastructure Engineer----->1.3436833371083934
Network Support Engineer----->1.3562936247886719
Sourcing Engineer----->1.3866618426751578
IT Security Engineer----->1.390747220031684
IT Security Engineer - Splunk----->1.390747220031684
Escalation Engineer----->1.3988435170161453
Telecommunication Engineer----->1.4052582783419612
Cockpit Engineer----->1.407882337212094
Su

In [14]:
Random_job_roles_from_web = [
    "Software Engineer",
    "Data Scientist",
    "Cybersecurity Analyst",
    "Registered Nurse",
    "Physician",
    "Medical Assistant",
    "Marketing Manager",
    "Accountant",
    "Project Manager",
    "Teacher",
    "Counselor",
    "Librarian",
    "Electrician",
    "Plumber",
    "Carpenter",
    "Graphic Designer",
    "Web Developer",
    "Content Writer",
    "Human Resources Specialist",
    "Management Analyst",
    "Mechanical Engineer",
    "Civil Engineer",
    "Chemical Engineer",
    "Environmental Engineer",
    "Sales Representative",
    "Customer Service Representative",
    "Business Analyst",
    "Financial Analyst",
    "Graphic Designer",
    "Web Developer",
    "User Experience (UX) Designer",
    "Data Analyst",
    "Software Tester",
    "Network Administrator",
    "System Administrator",
    "Marketing Specialist",
    "Social Media Manager",
    "Copywriter",
    "Editor",
    "Translator",
    "Interpreter",
    "Lawyer",
    "Paralegal",
    "Therapist",
    "Social Worker",
    "Architect",
    "Interior Designer",
    "Fashion Designer",
    "Web Content Manager",
    "Search Engine Optimization (SEO) Specialist",
    "Data Entry Clerk",
    "Administrative Assistant",
    "Receptionist",
    "Logistics Coordinator",
    "Supply Chain Manager",
]


for i in Random_job_roles_from_web:
    most_similar_job_role = find_most_similar_job_role(i, job_roles, job_role_embeddings, word2vec_model)
    print(f"{i} ---> {most_similar_job_role}")
    most_similar_job_role = find_most_similar_job_role_with_euclidean_distance(i, job_roles, job_role_embeddings, word2vec_model)
    print(f"{i} ---> {most_similar_job_role}")


Software Engineer ---> Embedded Software Engineer
Software Engineer ---> Embedded Software Engineer
Data Scientist ---> Data Scientist
Data Scientist ---> Data Scientist
Cybersecurity Analyst ---> Actuarial Analyst
Cybersecurity Analyst ---> Actuarial Analyst
Registered Nurse ---> Nurse
Registered Nurse ---> Nurse
Physician ---> Physician
Physician ---> Physician
Medical Assistant ---> Medical Office Assistant
Medical Assistant ---> Medical Office Assistant
Marketing Manager ---> Marketing Manager
Marketing Manager ---> Marketing Manager
Accountant ---> Accountant
Accountant ---> Accountant
Project Manager ---> Project Manager
Project Manager ---> Project Manager
Teacher ---> Educational Consultant
Teacher ---> Educational Consultant
Counselor ---> Genetic Counselor
Counselor ---> Genetic Counselor
Librarian ---> Modeling & Simulation Scientist
Librarian ---> Analytical Chemist
Electrician ---> Aircraft Electrician
Electrician ---> Aircraft Electrician
Plumber ---> Mechanic Helper
Plum